# Perceptron (Keras) — Zara Fashion Sales (Kaggle)

**Autor:** Nicollas Isaac Queiroz Batista
**Data:** 2025-08-17

## Objetivo
Treinar um **Perceptron** (um modelo de rede neural de camada única) em Keras para uma tarefa **binária** usando o dataset
**Zara Fashion Sales Dataset and Report** (Kaggle). O Perceptron, na prática, se comporta como uma **regressão logística** quando usamos uma única camada `Dense(1, activation="sigmoid")` — o que fornece uma **fronteira de decisão linear**. 

---

## Sobre o dataset 
Este conjunto reúne informações de produtos de moda vendidos pela Zara, incluindo **promoções**, **categorias**, **sazonalidade**, **volume de vendas**, **marca**, **preços** e metadados de coleta. Embora o número de linhas informado no Kaggle seja relativamente modesto (≈226 entradas), o **mix de colunas categóricas e numéricas**, além de possíveis **valores ausentes**, **skews** e **colinearidades** (ex.: preço, promoção, categoria) traz desafios reais para um modelo **linear** como o Perceptron.

**Campos esperados (amostra não exaustiva):**
- `promotion`, `Product Category`, `Seasonal`
- `Sales volume`, `brand`, `section`
- `Final price`, `Currency`
- `scraped Date`, `name`, `description` (podem conter lacunas/ruídos)

**Tarefa de classificação escolhida:** criar um rótulo binário **`high_revenue`** (1/0), onde `revenue = Final price * Sales volume`, e `high_revenue = 1` se `revenue` estiver **acima da mediana** do conjunto.  
Isso evita leakings óbvios (ex.: prever `promotion` a partir da própria presença de promoção) e cria uma tarefa de negócio plausível: **identificar produtos de alta receita** com base em atributos observáveis.

> **Observação importante:** por ser uma **fronteira de decisão linear**, o Perceptron serve como **baseline**. Espera-se desempenho modesto, pois relações não lineares (ex.: interações entre sazonalidade, categoria e preço) são comuns em varejo. Melhorias são discutidas ao final.

## 1) Setup e Carregamento de Dados

A seguir, usamos [`kagglehub`](https://github.com/Kaggle/kagglehub) para carregar o CSV diretamente do Kaggle.  
> **Dica:** Antes, configure suas credenciais do Kaggle (`~/.kaggle/kaggle.json`) ou siga as instruções do `kagglehub`.

In [1]:
# Se necessário, instale dependências no seu ambiente local:
# !pip install -q kagglehub[pandas-datasets] pandas numpy scikit-learn tensorflow
# (Opcional) Para limpar avisos menos relevantes
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd

# Keras / TensorFlow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Métricas de avaliação (pós-treino)
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Reprodutibilidade
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

import kagglehub
from kagglehub import KaggleDatasetAdapter

# Caminho do arquivo dentro do dataset Kaggle (ajuste se necessário após inspecionar a pasta)
# O dataset fornece um CSV; vamos tentar detectar automaticamente.
# Se souber o nome exato do CSV, defina aqui (ex.: "zara_fashion.csv").
file_path = ""

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "mohanz123/zara-fashion-sales-dataset-and-report",
    file_path,
)

print("Dimensões:", df.shape)
print(df.head(3))
print("\nColunas:", df.columns.tolist())

ModuleNotFoundError: No module named 'tensorflow'

## 2) Exploração inicial (EDA enxuta)

Verificamos: dimensões, tipos de dados, amostras, nulos e distribuição de algumas variáveis-chave.

In [ ]:
# Tipos e nulos
display(df.info())
print("\nNulos por coluna:")
print(df.isna().sum().sort_values(ascending=False))

# Amostras aleatórias
display(df.sample(min(5, len(df)), random_state=SEED))

# Estatísticas de colunas numéricas
display(df.select_dtypes(include=[np.number]).describe().T)

# Distribuições úteis (se existirem as colunas esperadas)
for col in ["promotion", "Product Category", "Seasonal", "brand", "section", "Currency"]:
    if col in df.columns:
        print(f"\nValue counts — {col}:")
        print(df[col].value_counts(dropna=False).head(10))

## 3) Criação do alvo binário `high_revenue`

- Calculamos `revenue = Final price * Sales volume` (tratando ausências/strings).
- Definimos `high_revenue = 1` se `revenue` for **maior que a mediana** do conjunto; caso contrário, `0`.

> Essa estratégia torna a tarefa menos trivial e mais próxima de um problema real de **ranking/propensão**.

In [ ]:
# Cópia de trabalho
data = df.copy()

# Normalização de nomes esperados
# Tentamos mapear variações de capitalização/espaços para garantir robustez.
def normalize_cols(cols):
    return {c: c.strip().lower().replace("  ", " ").replace(" ", "_") for c in cols}

rename_map = normalize_cols(data.columns)
data.rename(columns=rename_map, inplace=True)

# Possíveis nomes esperados
price_cols = [c for c in data.columns if "final" in c and "price" in c]
vol_cols = [c for c in data.columns if "sales" in c and "volume" in c]

if not price_cols or not vol_cols:
    raise ValueError(
        f"Não encontrei colunas para preço/volume. Preço detectado: {price_cols}, Volume detectado: {vol_cols}. "
        "Verifique o nome exato das colunas no CSV e ajuste o código."
    )

price_col = price_cols[0]
vol_col = vol_cols[0]

# Coerção para numérico
data[price_col] = pd.to_numeric(data[price_col], errors="coerce")
data[vol_col]   = pd.to_numeric(data[vol_col], errors="coerce")

# Remover linhas sem preço/volume
data = data.dropna(subset=[price_col, vol_col]).copy()

# Receita
data["revenue"] = data[price_col] * data[vol_col]

# Alvo binário por mediana
median_rev = data["revenue"].median()
data["high_revenue"] = (data["revenue"] > median_rev).astype(int)

print("Mediana de receita:", median_rev)
print("Balanceamento do alvo:")
print(data["high_revenue"].value_counts(normalize=True).rename({0:"classe 0", 1:"classe 1"}))

## 4) Seleção de features e pré-processamento

- **Entrada X:**
  - Numéricas: preço, volume e outras numéricas disponíveis.
  - Categóricas: `Product Category`, `Seasonal`, `brand`, `section`, `Currency` etc. (one-hot encoding).
- **Saída y:** `high_revenue` (0/1).

> O Perceptron exige features **numéricas**. Logo, convertemos as categóricas via **one-hot**. Opcionalmente padronizamos com `StandardScaler` para facilitar a otimização do `adam`.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

target = "high_revenue"

# Separar numéricas e categóricas
numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
# remover colunas que não devem ir para X
for drop_col in [target, "revenue"]:
    if drop_col in numeric_cols:
        numeric_cols.remove(drop_col)

categorical_cols = data.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numéricas:", numeric_cols[:10], "...")
print("Categóricas:", categorical_cols[:10], "...")

X = data[numeric_cols + categorical_cols].copy()
y = data[target].copy()

# Treino/teste estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# Transformador de colunas
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)

# Ajuste e transformação
X_train_proc = preprocess.fit_transform(X_train)
X_test_proc  = preprocess.transform(X_test)

print("Shape X_train_proc:", X_train_proc.shape)
print("Shape X_test_proc:", X_test_proc.shape)

## 5) Modelo Perceptron (Keras)

- **Arquitetura:** `Sequential([Dense(1, activation="sigmoid")])`
- **Otimizador:** `adam` — adapta taxas de aprendizado por parâmetro.
- **Loss:** `binary_crossentropy` — apropriada para classificação binária com `sigmoid`.
- **Métricas:** `accuracy` e **F1** (calculada após o treino com `sklearn` para robustez).

> Por ser apenas uma camada `Dense` com `sigmoid`, o modelo aprende um **hiperplano**: é o baseline mais simples e interpretável.

In [ ]:
# Construção do Perceptron
model = keras.Sequential([
    layers.Input(shape=(X_train_proc.shape[1],)),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

## 6) Treinamento

Treinamos por **50 épocas** com **batch_size=10**. Mantemos um `validation_split=0.2` apenas no conjunto de treino para monitorar overfitting.

In [ ]:
history = model.fit(
    X_train_proc, y_train.values,
    epochs=50,
    batch_size=10,
    validation_split=0.2,
    verbose=0
)

# Curvas (opcional)
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Binary Crossentropy")
plt.legend()
plt.title("Curva de perda")
plt.show()

plt.figure()
plt.plot(history.history["accuracy"], label="acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Curva de acurácia")
plt.show()

## 7) Avaliação no conjunto de teste

Geramos `y_pred` com `model.predict`, binarizamos com threshold padrão **0.5** e calculamos **Accuracy** e **F1**. 
Também exibimos **matriz de confusão** e `classification_report` para diagnóstico.

In [ ]:
# Predição
y_prob = model.predict(X_test_proc).ravel()
y_pred = (y_prob >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
f1  = f1_score(y_test, y_pred, zero_division=0)

print(f"Test Accuracy: {acc:.4f}")
print(f"Test F1:       {f1:.4f}")
print("\nMatriz de confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification report:")
print(classification_report(y_test, y_pred, zero_division=0))

## 8) Interpretação dos resultados

- **O que observar:** dado que o Perceptron usa **fronteira linear**, métricas medianas são esperadas quando a relação entre atributos e o rótulo é **não linear**.
- **Se F1 < Accuracy:** pode indicar **desbalanceamento** entre classes; o F1 pondera melhor **precisão/recall** para a classe positiva.
- **Curvas de validação:** divergência entre `loss` e `val_loss` sugere **overfitting**; use regularização ou menos features ruidosas.

### Possíveis Melhorias
1. **Engenharia de features:** interações (ex.: `price x seasonal`), bins de preço/volume, features de texto (TF‑IDF em `description`, `name`).  
2. **Tratamento de desbalanceamento:** `class_weight` no `model.fit` ou técnicas de amostragem (SMOTE).  
3. **Modelo não linear:** adicionar camadas ocultas (MLP) ou tentar árvores/boosting para capturar interações.  
4. **Validação robusta:** K‑fold estratificado e busca de hiperparâmetros.  
5. **Limpeza de dados:** remover outliers extremos e padronizar moedas/unidades.

> **Nota:** como baseline, o Perceptron é valioso pela **simplicidade** e por servir de **referência** para medir ganhos reais de modelos mais complexos.

## 9) (Opcional) Exportar arrays pré-processados

Caso queira treinar outros modelos rapidamente, é útil salvar `X_train_proc`, `X_test_proc`, `y_train`, `y_test`.

In [ ]:
# Salvar localmente (opcional)
# np.save("X_train_proc.npy", X_train_proc)
# np.save("X_test_proc.npy", X_test_proc)
# y_train.to_csv("y_train.csv", index=False)
# y_test.to_csv("y_test.csv", index=False)